In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET
import gpxpy
import gpxpy.gpx
import math
import matplotlib.pyplot as plt
import datetime as dt
import re
import matplotlib.dates as mdates
import folium
import japanize_matplotlib
from pytz import timezone
from shapely.geometry import Point
import json
import geojson
import os
import glob
from geojson import Point, Feature, FeatureCollection, dump, LineString, Polygon
import exifread

ModuleNotFoundError: No module named 'geopandas'

In [6]:
plt.style.use("ggplot")
date = dt.datetime(2024, 9, 14)
days = 1 #日数
mountain = "仙丈ヶ岳"

In [104]:
def main():
    track_to_csv(mountain, date)
    #marker_to_csv(mountain)
    mk_altitude_time_map(mountain, date, days)
    mk_altitude_distance_map(mountain, date, days)
    mk_distance_time_map(mountain, date, days)
    #make_track_map(mountain, date, days)
    pic_to_geojson()
    csv_to_geojson()

In [8]:
## 2点間の距離の計算
def cal_distance(lon1, lat1, lon2, lat2):
    R = 6371.0  # 地球の半径 (km)
    dlat = math.radians(lat2 - lat1)  # 緯度の差をラジアンに変換
    dlon = math.radians(lon2 - lon1)  # 経度の差をラジアンに変換
    # ハヴァサイン公式の計算
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R*c  #距離を返す

In [9]:
## マーカをCSVに変換
def marker_to_csv(mountain):
    ifile = "./01_data/{mountain}_marker.gpx".format(mountain=mountain)
    ofile = "./01_data/{mountain}_marker.csv".format(mountain=mountain)
    # タイムゾーン
    dt_tz = timezone('Asia/Tokyo')
    # 日時文字列形式
    dt_fmt = '%Y-%m-%d %H:%M:%S'
    # GPXファイルの読み込み
    gpx_file_r = open(ifile, 'r')
    # GPXファイルのパース
    gpx = gpxpy.parse(gpx_file_r)
    # GPXデータの読み込み
    #print(gpx)
    N = len(gpx.waypoints)
    lons = np.zeros(N); lats = np.zeros_like(lons); alts = np.zeros_like(lons)
    names = ["a"]*N; times = ["a"]*N
    #for point in gpx.waypoints:
    for i in range(N):
        lons[i] = gpx.waypoints[i].longitude
        lats[i] = gpx.waypoints[i].latitude
        alts[i] = gpx.waypoints[i].elevation
        names[i] = gpx.waypoints[i].name
        times[i] = gpx.waypoints[i].time.astimezone(dt_tz).strftime(dt_fmt)

    df = pd.DataFrame()
    df["longitude"] = lons; df["latitude"] = lats; df["altitude"] = alts; df["name"] = names; df["time"] = times
    df.to_csv(ofile, index=False, sep=" ")
    print("ifile", ifile)
    print("ofile", ofile)

In [10]:

## トラックをCSVに変換
def track_to_csv(mountain, date):
    ifile = "./01_data/{yyyy:04d}{mm:02d}{dd:02d}_{mountain}.gpx".format(yyyy=date.year, mm=date.month, dd=date.day, mountain=mountain)
    # タイムゾーン
    dt_tz = timezone('Asia/Tokyo')
    # 日時文字列形式
    dt_fmt = '%Y-%m-%d %H:%M:%S'
    # GPXファイルの読み込み
    gpx_file_r = open(ifile, 'r')
    # GPXファイルのパース
    gpx = gpxpy.parse(gpx_file_r)
    print(ifile)
    counts = 0
    for track in gpx.tracks:
        for segment in track.segments:
            points = segment.points
            #print(points.longitude)
            N = len(points)
            lons = np.zeros(N); lats = np.zeros_like(lons); alts = np.zeros_like(lons); dis = np.zeros_like(lons); times = ["a"]*N
            for i in range(N):
                lons[i] = points[i].longitude
                lats[i] = points[i].latitude
                alts[i] = points[i].elevation
                times[i] = points[i].time.astimezone(dt_tz).strftime(dt_fmt)

            for i in range(1, N):
                dis[i] = dis[i-1] + cal_distance(lons[i-1], lats[i-1], lons[i], lats[i])
    
            date = points[0].time.astimezone(dt_tz)
            ofile = "./01_data/{yyyy:04d}{mm:02d}{dd:02d}_{mountain}_track.csv".format(yyyy=date.year, mm=date.month, dd=date.day, mountain=mountain)
            df = pd.DataFrame()
            df["time"] = times; df["longitude"] = lons; df["latitude"] = lats; df["altitude"] = alts; df["distance"] = dis
            df.to_csv(ofile, index=False, sep=" ")
            print("ofile", ofile)
        counts += 1

In [11]:
## 高度-時間断面図
def mk_altitude_time_map(mountain, date, days=1):
    for i in range(days):
        dt_buf = date + dt.timedelta(days=i)
        ifile = "./01_data/{year:04d}{month:02d}{day:02d}_{mountain}_track.csv".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)
        ofile = "./02_map/{year:04d}{month:02d}{day:02d}_{mountain}_altitude-time_map.png".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)

        df = pd.read_csv(ifile, sep=" ")
        times = pd.to_datetime(df["time"])
        ts = pd.to_datetime(df["time"]).values
        alts = df["altitude"].values

        fig, ax = plt.subplots(figsize=(16, 9)) 

        ax.plot(ts, alts)
        ax.set_title("{year:04d}/{month:02d}/{day:02d}, {mountain}".format(mountain=mountain, year=dt_buf.year, month=dt_buf.month, day=dt_buf.day), fontsize=35, loc="left")
        ax.set_xlabel("time", fontsize=30)
        ax.set_ylabel("Altitude [m]", fontsize=30)
        ax.tick_params(labelsize=25)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
        plt.savefig(ofile)
        plt.close()
        print("ifile: ", ifile)
        print("ofile: ", ofile)

In [12]:
## 高度-距離断面図
def mk_altitude_distance_map(mountain, date, days=1):
    for i in range(days):
        dt_buf = date + dt.timedelta(days=i)
        ifile = "./01_data/{year:04d}{month:02d}{day:02d}_{mountain}_track.csv".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)
        ofile = "./02_map/{year:04d}{month:02d}{day:02d}_{mountain}_altitude-distance_map.png".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)

        df = pd.read_csv(ifile, sep=" ")
        xs = df["distance"].values
        alts = df["altitude"].values

        fig, ax = plt.subplots(figsize=(16, 9)) 

        ax.plot(xs, alts)
        ax.set_title("{year:04d}/{month:02d}/{day:02d}, {mountain}".format(mountain=mountain, year=dt_buf.year, month=dt_buf.month, day=dt_buf.day), fontsize=35, loc="left")
        ax.set_xlabel("Distance [km]", fontsize=30)
        ax.set_ylabel("Altitude [m]", fontsize=30)
        ax.tick_params(labelsize=25)
        plt.savefig(ofile)
        plt.close()

        print("ifile: ", ifile)
        print("ofile: ", ofile)


In [13]:
## 時間-距離断面図
def mk_distance_time_map(mountain, date, days=1):
    for i in range(days):
        dt_buf = date + dt.timedelta(days=i)
        ifile = "./01_data/{year:04d}{month:02d}{day:02d}_{mountain}_track.csv".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)
        ofile = "./02_map/{year:04d}{month:02d}{day:02d}_{mountain}_distance-time_map.png".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)

        df = pd.read_csv(ifile, sep=" ")
        xs = df["distance"].values
        ts = pd.to_datetime(df["time"]).values

        fig, ax = plt.subplots(figsize=(16, 9)) 

        ax.plot(ts, xs)
        ax.set_title("{year:04d}/{month:02d}/{day:02d}, {mountain}".format(mountain=mountain, year=dt_buf.year, month=dt_buf.month, day=dt_buf.day), fontsize=35, loc="left")
        ax.set_ylabel("Distance [km]", fontsize=30)
        ax.set_xlabel("Time", fontsize=30)
        ax.tick_params(labelsize=25)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
        plt.savefig(ofile)
        plt.close()

        print("ifile: ", ifile)
        print("ofile: ", ofile)

In [14]:
## XML -> Binary
def xml_to_bin(ifile, ofile):
    with open(ifile, "r") as f:
        # mesh
        r = re.compile("<mesh>(.+)</mesh>") ## 正規表現で見つけたい文字列を指定
        for ln in f:            ## lnには各行の文字列が入る
            m = r.search(ln)    ## 各行で検索
            #print(m)
            if m != None:
                mesh = m.group(1)   ## マッチした文字列の1つ目を取得
                #print(mesh)
                break

        # grid len
        r = re.compile("<gml:high>(.+) (.+)</gml:high>")
        for ln in f:
            m = r.search(ln)
            if m != None:
                xlen = int(m.group(1)) + 1
                ylen = int(m.group(2)) + 1
                N = xlen*ylen
                break
        # start
        r = re.compile("<gml:tupleList>")
        for ln in f:
            m = r.search(ln)
            if m != None:
                break
        # data
        data = np.zeros(N, dtype=np.float32)
        r  = re.compile("</gml:tupleList>")
        r2 = re.compile(",(.+)")
        count = 0
        for ln in f:
            m = r.search(ln)
            if m != None:
                break
            else:
                m = r2.search(ln)
                if m != None:
                    data[count] = float(m.group(1))
                    count = count + 1
    f.close()

    # Writing
    data = data.reshape(ylen, xlen)
    f = open(ofile, "wb")
    f.write(data)
    f.close()


In [15]:
## 等間隔（時間）で時間のピンを立てる
def make_track_map(mountain, date, days=1):
    ofile = "./02_map/{year:04d}{month:02d}{day:02d}_{mountain}_track.html".format(year=date.year, month=date.month, day=date.day, mountain=mountain)
    # 地図に線を追加する。緯度経度の配列をそのまま線として使う
    ## <<=========================>>
    ''' 1日目のデータ読み込み & 平均緯度経度を計算 '''
    i = 0
    dt_buf = date + dt.timedelta(days=i)
    ifile = "./01_data/{year:04d}{month:02d}{day:02d}_{mountain}_track.csv".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)
    df = pd.read_csv(ifile, sep=" ")
    print("ifile: ", ifile)
    lat_center = np.average(df["latitude"].values)
    lon_center = np.average(df["longitude"].values)
    locations = np.column_stack((df["latitude"].values, df["longitude"].values))
    ## <<=========================>>
    ''' 地図を描写 '''
    japan_map = folium.Map(location=[lat_center, lon_center],
                        tiles='https://cyberjapandata.gsi.go.jp/xyz/std/{z}/{x}/{y}.png',
                        attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">国土地理院</a>',
                        zoom_start=13)
    
    ## <<=========================>>
    ''' トラックをプロット & ピンを立てる '''
    line = folium.PolyLine(
        locations,
        color="red",
        weight=2.5,
        fill_opacity=0.2
    ).add_to(japan_map)

    for n in range(0, len(df["latitude"]), 3):
        marker = folium.Marker(
            location = locations[n], 
            popup = pd.to_datetime(df["time"][n]).strftime('%H:%M'),
            icon = folium.Icon(color='green')
        ).add_to(japan_map)
    ## <<=========================>>

    ''' 2日目以降の処理 '''
    for i in range(1, days):
        dt_buf = date + dt.timedelta(days=i)
        ifile = "./data/{year:04d}{month:02d}{day:02d}_{mountain}_track.csv".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)
        df = pd.read_csv(ifile, sep=" ")
        locations = np.column_stack((df["latitude"].values, df["longitude"].values))
        print("ifile: ", ifile)
        line = folium.PolyLine(
            locations,
            color="red",
            weight=2.5,
            fill_opacity=0.2
        ).add_to(japan_map)
        for n in range(0, len(df["latitude"]), 3):
            marker = folium.Marker(
                location = locations[n], 
                popup = pd.to_datetime(df["time"][n]).strftime('%H:%M'),
                icon = folium.Icon(color='green')
            ).add_to(japan_map)
    japan_map.save(ofile)

In [82]:

def read_exif_from_image(image_path):
    """画像ファイルからEXIFデータを読み込む"""
    try:
        with open(image_path, "rb") as f:
            exif_data = exifread.process_file(f)
        return exif_data
    except FileNotFoundError:
        print(f"ファイルが見つかりません: {image_path}")
        return None
    except IOError:
        print(f"ファイルの読み込みに失敗しました: {image_path}")
        return None

        
def dms_to_decimal(dms):
    """度分秒（DMS）リストから10進数の座標に変換する関数"""
    degrees, minutes, seconds = dms
    return degrees + (minutes / 60) + (seconds / 3600)


def get_coordinates(exif_data):
    """EXIFタグから緯度と経度を抽出してタプルで返す"""
    try:
        pre_latitude = eval(exif_data["GPS GPSLatitude"].printable)
        pre_longitude = eval(exif_data["GPS GPSLongitude"].printable)
        datetime = exif_data['EXIF DateTimeDigitized'].printable
        alt = eval(exif_data['GPS GPSAltitude'].printable)
        latitude = dms_to_decimal(pre_latitude)
        longitude = dms_to_decimal(pre_longitude)
        return (datetime, latitude, longitude, alt)
    except KeyError:
        print("必要なGPS情報が含まれていません。")
        return None


def pic_to_geojson():
    ofile = "./data/img_info.geojson"
    pictures = glob.glob("./img/*.jpg")
    n = len(pictures)
    datetimes = ["date"]*n
    lats = np.zeros(n)
    lons = np.zeros(n)
    alts = np.zeros(n)

    for i in range(n):
        image_path = pictures[i]
        exif_data = read_exif_from_image(image_path)
        if exif_data:
            result = get_coordinates(exif_data)
            if result:
                #print("日時，緯度，経度，標高:", result)
                datetimes[i], lats[i], lons[i], alts[i] = result
                #datetime, lat , lon, alt = result
            else:
                print("GPS情報の取得に失敗しました。")
                os.exit(1)
        else:
            print("EXIFデータの読み込みに失敗しました。")
            os.exit(1)

    geometry = [  Point(xy) for xy in zip(lons, lats)]
    df = pd.DataFrame()
    df["datetime"] = datetimes
    df["altitude"] = alts
    df["name"] = pictures

    geo_df = gpd.GeoDataFrame(df, geometry=geometry)
    geo_df.to_file(ofile, driver='GeoJSON', encoding='utf-8')
    print("ofile:", ofile)


pic_to_geojson()

0
1
2
3
4
5
6
ofile: ./data/img_info.geojson


In [1]:
def csv_to_geojson():
    ifiles = glob.glob("./data/*_track.csv")
    all_df = pd.DataFrame()
    ofile = "./data/track.geojson"
    total_distance = 0
    for ifile in ifiles:
        each_df = pd.read_csv(ifile, sep=" ")
        #total_distance  += each_df["distance"][-1]
        all_df = pd.concat([all_df, each_df])

    xy = [xy for xy in zip(all_df["longitude"], all_df["latitude"])]
    line = LineString(xy)
    start_time = all_df["time"][0]
    end_time = all_df["time"].values[-1]
    max_altitude = np.max(all_df["altitude"])
    min_altitude = np.min(all_df["altitude"])

    features = []
    feature = {
        "type": "Feature",
        "properties": {
            "start_time": str(start_time),
            "end_time": str(end_time),
            "max_altitude": str(max_altitude),
            "min_altitude": str(min_altitude),
            "total_distance": str(total_distance),
        },
        "geometry": {
            "type": "LineString",
            "coordinates": xy,
        },
    }


    features.append(feature)

    # GeoJSONファイルを書き込む
    with open(ofile, "w", encoding='utf_8') as f:
        json.dump({"type":"FeatureCollection", "features": features}, f, indent = 2, ensure_ascii=False)

csv_to_geojson()

NameError: name 'glob' is not defined

In [101]:
if __name__ == "__main__":
    main()

NameError: name 'main' is not defined